In [ ]:
# %% [markdown]
# # Voltage Time Series Analysis
# 
# This notebook performs comprehensive analysis of voltage time series data including:
# - Visualization with multiple moving averages
# - Peak and trough detection
# - Threshold analysis
# - Slope acceleration detection

# %%
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks
import os
from datetime import datetime

# Constants
DATA_FILE = 'Sample_Data.csv'
VOLTAGE_THRESHOLD = 20
SLOPE_ACCEL_THRESHOLD = -5
MA_WINDOW_5 = 5
MA_WINDOW_1000 = 1000
MA_WINDOW_5000 = 5000

# %% [markdown]
# ## Data Loading and Preparation

# %%
# Check file exists
if not os.path.exists(DATA_FILE):
    raise FileNotFoundError(f"Data file {DATA_FILE} not found")

# Load data
df = pd.read_csv(DATA_FILE)

# Convert timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Check for invalid timestamps
invalid_timestamps = df['Timestamp'].isna()
if invalid_timestamps.any():
    print(f"Warning: {invalid_timestamps.sum()} rows with invalid timestamps will be dropped")
    df = df.dropna(subset=['Timestamp'])

print(f"Data loaded successfully with {len(df)} records")
print(f"Time range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")

# %% [markdown]
# ## Data Visualization with Moving Averages

# %%
# Calculate moving averages
df['MA5'] = df['Values'].rolling(window=MA_WINDOW_5).mean()
df['MA1000'] = df['Values'].rolling(window=MA_WINDOW_1000).mean()
df['MA5000'] = df['Values'].rolling(window=MA_WINDOW_5000).mean()

# Create interactive plot
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(
    x=df['Timestamp'], y=df['Values'],
    mode='lines', name='Original Values',
    line=dict(color='blue', width=1.5)
))

fig.add_trace(go.Scatter(
    x=df['Timestamp'], y=df['MA5'],
    mode='lines', name=f'{MA_WINDOW_5}-Day MA',
    line=dict(color='orange', width=1.5, dash='dot')
))

fig.add_trace(go.Scatter(
    x=df['Timestamp'], y=df['MA1000'],
    mode='lines', name=f'{MA_WINDOW_1000}-Point MA',
    line=dict(color='red', width=1.5)
))

fig.add_trace(go.Scatter(
    x=df['Timestamp'], y=df['MA5000'],
    mode='lines', name=f'{MA_WINDOW_5000}-Point MA',
    line=dict(color='green', width=1.5)
))

# Update layout
fig.update_layout(
    title='Voltage with Multiple Moving Averages',
    xaxis_title='Timestamp',
    yaxis_title='Voltage',
    legend_title='Legend',
    hovermode='x unified',
    template='plotly_white',
    height=600,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1d', step='day', stepmode='backward'),
                dict(count=7, label='1w', step='day', stepmode='backward'),
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)

fig.show()

# %% [markdown]
# ## Peak and Trough Detection

# %%
# Find peaks and lows
peaks, _ = find_peaks(df['Values'], prominence=5)  # Added prominence for better detection
lows, _ = find_peaks(-df['Values'], prominence=5)

# Create DataFrames
peaks_df = df.iloc[peaks][['Timestamp', 'Values']].copy()
lows_df = df.iloc[lows][['Timestamp', 'Values']].copy()

# Add rank columns
peaks_df['Rank'] = peaks_df['Values'].rank(ascending=False)
lows_df['Rank'] = lows_df['Values'].rank()

print(f"Found {len(peaks_df)} peaks and {len(lows_df)} lows")

# Visualization
fig_peaks = go.Figure()

fig_peaks.add_trace(go.Scatter(
    x=df['Timestamp'], y=df['Values'],
    mode='lines', name='Voltage',
    line=dict(color='blue', width=1)
))

fig_peaks.add_trace(go.Scatter(
    x=peaks_df['Timestamp'], y=peaks_df['Values'],
    mode='markers', name='Peaks',
    marker=dict(color='green', size=8, symbol='triangle-up')
))

fig_peaks.add_trace(go.Scatter(
    x=lows_df['Timestamp'], y=lows_df['Values'],
    mode='markers', name='Lows',
    marker=dict(color='red', size=8, symbol='triangle-down')
))

fig_peaks.update_layout(
    title='Peak and Trough Detection',
    xaxis_title='Timestamp',
    yaxis_title='Voltage',
    template='plotly_white'
)

fig_peaks.show()

# Display tables
print("\nTop 5 Peaks:")
display(peaks_df.sort_values('Values', ascending=False).head(5))

print("\nTop 5 Lows:")
display(lows_df.sort_values('Values').head(5))

# %% [markdown]
# ## Voltage Threshold Analysis

# %%
# Find values below threshold
below_threshold = df[df['Values'] < VOLTAGE_THRESHOLD]

print(f"Found {len(below_threshold)} instances where voltage < {VOLTAGE_THRESHOLD}")

# Visualization
fig_threshold = go.Figure()

fig_threshold.add_trace(go.Scatter(
    x=df['Timestamp'], y=df['Values'],
    mode='lines', name='Voltage',
    line=dict(color='blue', width=1)
))

fig_threshold.add_trace(go.Scatter(
    x=below_threshold['Timestamp'], y=below_threshold['Values'],
    mode='markers', name=f'Below {VOLTAGE_THRESHOLD}',
    marker=dict(color='orange', size=6)
))

# Add threshold line
fig_threshold.add_hline(
    y=VOLTAGE_THRESHOLD,
    line_dash="dot",
    annotation_text=f"Threshold: {VOLTAGE_THRESHOLD}",
    annotation_position="bottom right",
    line_color="red"
)

fig_threshold.update_layout(
    title=f'Voltage Below {VOLTAGE_THRESHOLD}',
    xaxis_title='Timestamp',
    yaxis_title='Voltage',
    template='plotly_white'
)

fig_threshold.show()

# Display table
if not below_threshold.empty:
    print("\nAll instances below threshold:")
    display(below_threshold[['Timestamp', 'Values']])
else:
    print("No instances found below threshold")

# %% [markdown]
# ## Downward Slope Acceleration Analysis

# %%
# Calculate slope and slope change
df['Slope'] = df['Values'].diff()
df['Slope_Change'] = df['Slope'].diff()

# Find accelerating downward slopes
accelerated_slopes = df[(df['Slope'] < 0) & (df['Slope_Change'] < SLOPE_ACCEL_THRESHOLD)]

print(f"Found {len(accelerated_slopes)} instances of downward slope acceleration")

# Visualization
fig_slope = go.Figure()

fig_slope.add_trace(go.Scatter(
    x=df['Timestamp'], y=df['Values'],
    mode='lines', name='Voltage',
    line=dict(color='blue', width=1)
))

fig_slope.add_trace(go.Scatter(
    x=accelerated_slopes['Timestamp'], y=accelerated_slopes['Values'],
    mode='markers', name='Slope Acceleration',
    marker=dict(color='purple', size=8, symbol='x')
))

fig_slope.update_layout(
    title='Downward Slope Acceleration Points',
    xaxis_title='Timestamp',
    yaxis_title='Voltage',
    template='plotly_white'
)

fig_slope.show()

# Display table
if not accelerated_slopes.empty:
    print("\nSlope acceleration details:")
    display(accelerated_slopes[['Timestamp', 'Values', 'Slope', 'Slope_Change']])
else:
    print("No slope acceleration instances found")

# %% [markdown]
# ## Summary Statistics

# %%
# Basic statistics
print("Overall Statistics:")
display(df['Values'].describe().to_frame())

# Time-based statistics
print("\nStatistics by Hour (if available):")
df['Hour'] = df['Timestamp'].dt.hour
hourly_stats = df.groupby('Hour')['Values'].agg(['mean', 'min', 'max', 'std'])
display(hourly_stats)

# %% [markdown]
# ## Export Results

# %%
# Save all results to Excel
output_file = 'voltage_analysis_results.xlsx'

with pd.ExcelWriter(output_file) as writer:
    df.to_excel(writer, sheet_name='Full Data', index=False)
    peaks_df.to_excel(writer, sheet_name='Peaks', index=False)
    lows_df.to_excel(writer, sheet_name='Lows', index=False)
    below_threshold.to_excel(writer, sheet_name='Below Threshold', index=False)
    accelerated_slopes.to_excel(writer, sheet_name='Slope Acceleration', index=False)
    hourly_stats.to_excel(writer, sheet_name='Hourly Stats')

print(f"All results saved to {output_file}")